In [8]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key="",
    api_version="2024-02-01",
    azure_endpoint = "https://jlaopenAI.openai.azure.com/"
    )

In [9]:
import requests
import json

#Generate Embeddings
def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

#RAG Method to get the answer from AI search index
def get_your_data(userQuestion):
    
    searchServiceName = "cogsearchjla";
    apiKey = "";
    indexName = "brochures-index";
    apiVersion = "2023-11-01";
    vectorField = "contentVector"


    print("Python Function: 'get_your_data' was triggered.\nNow calling external api.. (Azure AI Search)")

    # Using the Ada model for embedding
    embedding = generate_embeddings(userQuestion)

    # Construct the API endpoint
    url = f"https://{searchServiceName}.search.windows.net/indexes/{indexName}/docs/search?api-version={apiVersion}"

    # Set the request headers
    headers = {
        "Content-Type": "application/json",
        "api-key": apiKey
    }

    # Set the request body
    body = {
        "count": True,
        "search": userQuestion,
        "select": "content, url, title",
        #"filter": "geo.distance(Location, geography'POINT(-77.03241 38.90166)') le 300",
        #"facets": ["Address/StateProvince"],
        "vectorQueries": [
            {
                "vector": embedding,
                "kind": "vector",
                "k": 2, #results for vector-only queries
                "fields": "contentVector"
            }
        #     # ,
        #     # {
        #     #     "value": [<array of embeddings>],
        #     #     "k": 7,
        #     #     "fields": "Description_frVector"
        #     # }
        ],
        "top": 2, #results for hybrid queries that include a "search" parameter
        "highlight": "content,title", #highlight phrase keyword in the content field
        "highlightPreTag": "<strong>",
        "highlightPostTag": "</strong>",
        # "queryType": "semantic",
        # "semanticConfiguration": "my-semantic-config"
    }

    # Make the POST request
    response = requests.post(url, headers=headers, json=body)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        response_data = response.json()
        # print("Python Function: 'get_your_data' finished and provided following value: \n" + json.dumps(response_data))
    else:
        print("Failed to retrieve data.")

    return response_data

get_your_data("what can i do in dubai?")

Python Function: 'get_your_data' was triggered.
Now calling external api..


{'@odata.context': "https://cogsearchjla.search.windows.net/indexes('brochures-index')/$metadata#docs(*)",
 '@odata.count': 6,
 'value': [{'@search.score': 0.03333333507180214,
   '@search.highlights': {'content': ['Title: <strong>Dubai</strong> Brochure.pdfMargie’s Travel Presents…  \n<strong>Dubai</strong>  \n \n \n<strong>Dubai</strong> is the largest and most populous city <strong>in</strong> the United \nArab Emirates.',
     "Abu \nDhabi and <strong>Dubai</strong> are the only two emirates to have veto \npower over critical matters of national importance <strong>in</strong> the \ncountry's Federal Supreme Council.",
     "The city of <strong>Dubai</strong> is \nlocated on the emirate's northern coastline and heads the \nDuba <strong>i</strong>-Sharjah -Ajman metropolitan area.",
     '<strong>Dubai</strong>  Hotels  \nMargie’s Travel offers the following accommodation \noptions <strong>in</strong> <strong>Dubai</strong> : \nThe Creek  Hotel  \nFriendly boutique hotel with <strong

In [10]:
import openai
import json

#Custom functions that can be called by the LLM. Providing insights into the arguments that are required for the function to be called.
custom_functions = [
    {
        'name': 'get_your_data',
        'description': 'Get travel advise from our travel catalogues using azure ai',
        'parameters': {
            'type': 'object',
            'properties': {
                'userQuestion': {
                    'type': 'string',
                    'description': 'Qustion asked by the user'
                }
            },
            "required": ["userQuestion"]
        }
    }
]
#Configuration of the LLM
def Call_LLM_Agent(chat_history):
    return (client.chat.completions.create(
        model='gpt-4o', # deployment_id could be one of {gpt-35-turbo, gpt-35-turbo-16k}
        messages=chat_history,
        functions = custom_functions,
        function_call = 'auto',
        temperature=0)
    )

#Instructions to the LLM for how to execute a "Tool", and log the actions and results between user, tool, and llm to provide the LLM context of what has happend so far
def LLM_logic(response, chat_history):
    #Getting assistant message 
    response_message = response.choices[0].message

    # Call python function
    if dict(response_message).get('function_call'):
            
        # Which function call was invoked
        function_called = response_message.function_call.name
        
        # Extracting the arguments
        function_args  = json.loads(response_message.function_call.arguments)
        
        # Function names
        available_functions = {
            "get_your_data": get_your_data
        }
        
        #determine which function to call
        fuction_to_call = available_functions[function_called]

        # append assistant's to history that we call the function and the passed arguments
        chat_history.append({"role": "assistant", "content": "null", "function_call": {"name": "" + function_called + "", "arguments": "" + json.dumps(function_args)}})

        # execute the function, and get return value from function
        custom_function_response = fuction_to_call(*list(function_args .values()))

        # append assistant's to history the result/response from the custom function
        chat_history.append({"role": "function", "name": function_called, "content": json.dumps(custom_function_response)})
        
        #after function has finished, and delivered result back to the chat history, assistant looks at chat history and provides a response to the user.
        final_message = Call_LLM_Agent(chat_history)
        final_message = final_message.choices[0].message.content
    else:
        #if not using function calling, return the llm response to user
        final_message = response_message.content

    print("\nFinal message:\n")
    print(final_message)


In [26]:
#Start demo1: Example demonstrates LLM will choose the right tool to assign a hotel room to the user based on the conversation.
chat_history1=[
        {"role": "system", "content": "You are a helpful assistant providing travel tips to users. You have access to a travel catalogue. You provide answers in bullet form. Each bullet will have a number as a reference to the document source that is provided by the URL field. e.g. [1]. If you dont know the source, assign the source 'pretrained knowledge'.  List these source at the end of the message."},
        {"role": "user", "content": "My name is Jeffrey Lai"},
        {"role": "assistant", "content": "Any place you want to visit or special things you want to experience?"},
        {"role": "user", "content": "Dubai."},
    ]

response1 = Call_LLM_Agent(chat_history1)
LLM_logic(response1, chat_history1)

Python Function: 'get_your_data' was triggered.
Now calling external api..

Final message:

Here are some top attractions and accommodations in Dubai:

### Top Attractions:
1. **Burj Khalifa**: The tallest building in the world, offering stunning views from its observation decks. [pretrained knowledge]
2. **Dubai Mall**: One of the largest shopping malls globally, featuring an aquarium, ice rink, and numerous dining options. [pretrained knowledge]
3. **Dubai Creek**: A historic area with traditional markets (souks) and boat rides. [1]
4. **Palm Jumeirah**: An artificial archipelago with luxury hotels, restaurants, and beaches. [pretrained knowledge]
5. **Dubai Marina**: A modern district with skyscrapers, restaurants, and a beautiful waterfront. [pretrained knowledge]
6. **The Lost City Hotel**: Luxurious accommodation with an onsite waterpark and aquarium. [1]

### Accommodation Options:
1. **The Creek Hotel**: A friendly boutique hotel in the heart of the bustling Dubai Creek area. [